In [1]:
import pandas as pd
from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report

In [20]:
test_true_ratio = pd.read_csv('data/transcript_balanced_test.csv')
# test_balanced_ratio = pd.read_csv('data/test_data_new.csv')
test_true_ratio = test_true_ratio.sample(n=100000)

In [18]:
train = pd.read_csv('data/transcript_balanced_train.csv')

## Note: only 10000 is taken from train
This is to reduce the training time

10000 takes less than 1 min

100000 takes ~ 10 min

1000000 takes ~ 10 hours

runned using 12100F

In [21]:
train = train.sample(n=100000)

In [22]:
x_train = train.drop(['label'], axis=1)
y_train = train['label']

In [23]:
x_train.head()

,transcript_position,-1_flank_length,-1_flank_std,-1_flank_mean,central_length,central_std,central_mean,+1_flank_length,+1_flank_std,+1_flank_mean,...,1-mer_2,1-mer_3,1-mer_4,1-mer_5,1-mer_6,5-mer_0,5-mer_1,5-mer_2,new_transcript_id,new_gene_id
1286551,0.024296,0.056169,0.014015,0.578005,0.065802,0.045531,0.565217,0.061892,0.024827,0.300000,...,1.0,0.0,0.0,0.0,0.666667,0.913043,0.529412,0.608696,0.594734,0.558699
1729167,0.073370,0.079580,0.010760,0.488491,0.073931,0.018610,0.442935,0.025492,0.012358,0.438961,...,0.0,0.0,0.0,1.0,1.000000,0.521739,0.117647,0.478261,0.832427,0.385646
1829830,0.095477,0.029553,0.027034,0.332481,0.035320,0.074053,0.673913,0.038653,0.012681,0.348052,...,1.0,0.0,0.0,1.0,1.000000,0.217391,1.000000,1.000000,0.915793,0.319441
99445,0.045656,0.026794,0.014646,0.398977,0.150377,0.046160,0.633152,0.027508,0.013263,0.371429,...,1.0,0.0,0.0,1.0,0.666667,1.000000,1.000000,0.956522,0.351108,0.749196
797881,0.023389,0.106284,0.013432,0.641944,0.086511,0.056906,0.660326,0.055134,0.012165,0.270130,...,1.0,0.0,0.0,0.0,0.333333,0.826087,0.176471,0.565217,0.522050,0.235171


In [24]:
y_train.astype('int')

1286551    0
1729167    0
1829830    0
99445      0
797881     0
          ..
121077     0
808189     0
1340232    0
1402288    0
804101     0
Name: label, Length: 100000, dtype: int32

In [25]:
y_train.head()

1286551    0.0
1729167    0.0
1829830    0.0
99445      0.0
797881     0.0
Name: label, dtype: float64

In [35]:
from sklearn.svm import SVC

# Set equal costs for both types of misclassifications
C01 = 1  # Cost of classifying true negatives as true positives
C10 = 1  # Cost of classifying true positives as true negatives

# Initialize the SVM classifier with the cost matrix
svm = SVC(class_weight='balanced')

In [39]:
svm.fit(x_train, y_train)

SVC(class_weight='balanced')

In [40]:
import joblib

# Save the trained SVM model to a file
model_filename = 'svm_model_with_transcript_balanced_data_position_100000_F1_optimize.pkl'
joblib.dump(svm, model_filename)

# # Later, to load the saved model
# loaded_model = joblib.load(model_filename)

# # You can now use `loaded_model` for predictions or other purposes

['svm_model_with_transcript_balanced_data_position_100000_F1_optimize.pkl']

## classification report using balanced test dataset

In [41]:
# x_test = test_balanced_ratio.drop(["label"], axis=1)
# y_test = test_balanced_ratio["label"]

In [42]:
# y_pred = clf.predict(x_test)

In [43]:
# Calculate accuracy
# accuracy = accuracy_score(y_test, y_pred)
# print(f'Accuracy: {accuracy:.2f}')

In [44]:
# Display a classification report
# class_report = classification_report(y_test, y_pred)
# print('Classification Report:\n', class_report)

## classification report using true test dataset

In [45]:
x_test = test_true_ratio.drop(["label"], axis=1)
y_test = test_true_ratio["label"]

In [46]:
y_pred = svm.predict(x_test)

In [47]:
# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.71


In [48]:
# Display a classification report
class_report = classification_report(y_test, y_pred)
print('Classification Report:\n', class_report)

Classification Report:
               precision    recall  f1-score   support

         0.0       0.98      0.70      0.82     95099
         1.0       0.12      0.79      0.21      4901

    accuracy                           0.71    100000
   macro avg       0.55      0.75      0.51    100000
weighted avg       0.94      0.71      0.79    100000

